In [ ]:
%tensorflow_version 2.x

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
import tensorflow as tf

This particular data set has 3 classes of flowers ( Setosa, Versicolor, Virginica) based on information on (sepal length & width and petal length & width). We are going to highlight the column names and the species.

In [ ]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [ ]:
training_path = tf.keras.utils.get_file('iris_training.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv')
test_path = tf.keras.utils.get_file('iris_test.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv')

training = pd.read_csv(training_path, names = CSV_COLUMN_NAMES, header = 0)
test = pd.read_csv(test_path, names = CSV_COLUMN_NAMES, header = 0)

y_training = training.pop('Species')
y_test = test.pop('Species')

Input function time!

In [ ]:
def input_fn(features, labels, shuffle = True, batch_size = 256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  if shuffle:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)

feature column

In [ ]:
my_feature_column = []
for key in training.keys():
  my_feature_column.append(tf.feature_column.numeric_column(key=key))

# Building the Model

In [ ]:
#We are building a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns = my_feature_column,
    hidden_units = [30, 10],
    n_classes = 3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpp6c8nk9y', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Training the Model


In [ ]:
classifier.train(input_fn = lambda: input_fn(training, y_training, shuffle = True), steps = 12000)
# Steps is simlar to epoch but for epoch it would go through the dataset 12000 times, for step it keeps going through the dataset till it has seen 12000 objects.

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpp6c8nk9y/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.1091018, step = 0
INFO:tensorflow:global_step/sec: 509.731
INFO:tensorflow:loss = 1.0075092, step = 100 (0.200 sec)
INFO:tensorflow:global_step/sec: 675.819
INFO:tensorflow:loss = 0.8965778, step = 200 (0.148 sec)
INFO:tensorflow:global_step/sec

# Evaluate

In [ ]:
eval_result = classifier.evaluate(input_fn = lambda: input_fn(test, y_test, shuffle = False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-12-28T18:12:59
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpp6c8nk9y/model.ckpt-12000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.23222s
INFO:tensorflow:Finished evaluation at 2021-12-28-18:13:00
INFO:tensorflow:Saving dict for global step 12000: accuracy = 0.96666664, average_loss = 0.26218498, global_step = 12000, loss = 0.26218498
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 12000: /tmp/tmpp6c8nk9y/model.ckpt-12000

Test set accuracy: 0.967



# Prediction for Data in Eval

In [ ]:
my_result = list(classifier.predict(input_fn = lambda: input_fn(test, y_test, shuffle = False)))

i = 0
while i < len(my_result):
  print(test.loc[i]) 
  print(y_test.loc[i])
  class_id = my_result[i]['class_ids'][0]
  probability = my_result[i]['probabilities'][class_id]
  print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100* probability))
  i = i + 5

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpp6c8nk9y/model.ckpt-12000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
SepalLength    5.9
SepalWidth     3.0
PetalLength    4.2
PetalWidth     1.5
Name: 0, dtype: float64
1
Prediction is "Versicolor" (67.5%)
SepalLength    6.2
SepalWidth     2.9
PetalLength    4.3
PetalWidth     1.3
Name: 5, dtype: float64
1
Prediction is "Versicolor" (76.1%)
SepalLength    7.1
SepalWidth     3.0
PetalLength    5.9
PetalWidth     2.1
Name: 10, dtype: float64
2
Prediction is "Virginica" (83.1%)
SepalLength    5.1
SepalWidth     3.5
PetalLength    1.4
PetalWidth     0.2
Name: 15, dtype: float64
0
Prediction is "Setosa" (97.6%)
SepalLength    5.4
SepalWidth     3.4
PetalLength    1.7
PetalWidth     0.2
Name: 20, dtype: float64
0
Prediction is "Setosa" (96.3%)
SepalLength    6.1
SepalWidth     3.0
PetalLength   

# Prediction for Input Data

In [ ]:
def input_fn(features, batch_size = 256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

for feature in features:
  valid = True
  while valid:
    val = input(feature + ': ')
    if not val.isdigit():
      valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn = lambda: input_fn(predict))
for pre_dict in predictions:
  class_id = pre_dict['class_ids'][0]
  probability = pre_dict['probabilities'][class_id]

  print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100* probability))

SepalLength: 6.9
SepalWidth: 3.1
PetalLength: 5.4
PetalWidth: 2.1
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpp6c8nk9y/model.ckpt-12000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Virginica" (74.4%)
